## Importing Libraries

In [ ]:
import shutil
shutil.rmtree("/content/sample_data")

In [ ]:
! pip install mediapipe
! pip install sk-video
! pip install keras-video-generators
import numpy as np 
import pandas as pd 
import os
import skvideo.io
import cv2 
import mediapipe as mp
import time
import matplotlib.pyplot as plt
import glob
import keras
from keras_video import VideoFrameGenerator
# use sub directories names as classes
import tensorflow
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM,Flatten, TimeDistributed, Conv2D, Dropout,Input

     |████████████████████████████████| 47.4MB 115kB/s 
     |████████████████████████████████| 2.3MB 6.4MB/s 


In [ ]:
shutil.rmtree("/content/sample_data")

## Downloading and Unzipping Data

In [ ]:
! wget https://zenodo.org/record/4010759/files/Greetings_1of2.zip
! wget https://zenodo.org/record/4010759/files/Greetings_2of2.zip

--2021-05-01 12:34:37--  https://zenodo.org/record/4010759/files/Greetings_1of2.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1573733886 (1.5G) [application/octet-stream]
Saving to: ‘Greetings_1of2.zip’

Greetings_1of2.zip  100%[===================>]   1.46G  18.3MB/s    in 1m 45s  

2021-05-01 12:36:24 (14.2 MB/s) - ‘Greetings_1of2.zip’ saved [1573733886/1573733886]

--2021-05-01 12:36:24--  https://zenodo.org/record/4010759/files/Greetings_2of2.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1208700458 (1.1G) [application/octet-stream]
Saving to: ‘Greetings_2of2.zip’

Greetings_2of2.zip  100%[===================>]   1.12G  18.6MB/s    in 89s     

2021-05-01 12:37:54 (13.0 MB/s) - ‘Greetings_2of2.zip’ saved [1208700458/1

In [ ]:
! unzip Greetings_1of2.zip
! unzip Greetings_2of2.zip

In [ ]:
! wget https://zenodo.org/record/4010759/files/Pronouns_1of2.zip
! wget https://zenodo.org/record/4010759/files/Pronouns_2of2.zip

--2021-05-01 12:38:25--  https://zenodo.org/record/4010759/files/Pronouns_1of2.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1419853451 (1.3G) [application/octet-stream]
Saving to: ‘Pronouns_1of2.zip’

Pronouns_1of2.zip   100%[===================>]   1.32G  4.82MB/s    in 2m 20s  

2021-05-01 12:40:46 (9.68 MB/s) - ‘Pronouns_1of2.zip’ saved [1419853451/1419853451]

--2021-05-01 12:40:46--  https://zenodo.org/record/4010759/files/Pronouns_2of2.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 946837765 (903M) [application/octet-stream]
Saving to: ‘Pronouns_2of2.zip’

Pronouns_2of2.zip   100%[===================>] 902.97M  10.7MB/s    in 66s     

2021-05-01 12:41:54 (13.7 MB/s) - ‘Pronouns_2of2.zip’ saved [946837765/946837765

In [ ]:
!unzip Pronouns_1of2.zip
!unzip Pronouns_2of2.zip

In [ ]:
shutil.rmtree("/content/Pronouns/46. you (plural)")
shutil.rmtree("/content/Pronouns/44. it")

In [ ]:
os.makedirs("/content/Input")

### Adding our Own Data

In [ ]:
wordtype=["Greetings","Pronouns"]

for i in wordtype:
   for dirpath,dirfile,filename in os.walk(os.path.join("/content",i)):
     print(dirfile)

['50. Alright', '55. Thank you', '56. Pleased', '53. Good evening', '54. Good night', '48. Hello', '51. Good Morning', '52. Good afternoon', '49. How are you']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['45. we', '42. he', '43. she', '47. they', '40. I', '41. you']
[]
[]
[]
[]
['.ipynb_checkpoints']
[]
[]


In [ ]:
d=['/content/Greetings',"/content/Pronouns/"]
target_dir = '/content/Input'

for i in d:
    file_names = os.listdir(i)
    for file_name in file_names:
        shutil.move(os.path.join(i, file_name), target_dir)

In [ ]:
os.makedirs("/content/Output")

## Initializing Holistic Model

In [ ]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic

def pose_estimation(image,results):
        
        # 1. Draw face landmarks
        #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
        #                         mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
        #                         mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
        #                         )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(102,255,51), thickness=3, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(102,255,51), thickness=3, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(255,255,0), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2)
                                 )
                        
        return image

NameError: ignored

## Applying Holistic Model on Video and Resizing it.

In [ ]:
def video_array_maker(pather,height=224,width=224):
  videodata = skvideo.io.vread(pather)  
  outpath=os.path.join("./Output",pather.split("/")[3],os.path.split(pather)[1])
  out = cv2.VideoWriter(outpath,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (width,height))
  #start=time.time()

  with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#    arr=[]
    actualframe=len(videodata)
    #print("Actual frame {}".format(actualframe))
    if actualframe >=45:
          for i in range (actualframe):
            x=round (actualframe/(45)  * i)
            if x >=actualframe:
                    break
            else:
                frame =videodata[x]
                #frame =cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA)
                results = holistic.process(frame)  

                output = pose_estimation(frame,results)
                output =cv2.resize(output,(width,height),interpolation=cv2.INTER_AREA)
                output =cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
#                arr.append(output)
                out.write(output)               
    else:
          for i in range(actualframe):
              frame=videodata[i]
              frame=cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA)
              results = holistic.process(frame)  
              output = pose_estimation(frame,results)
              output =cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
              out.write(output)
          for i in range(45-actualframe):
              
              newframe=np.zeros(shape=(height,width,3))
              

              out.write(np.uint8(newframe))
    out.release()
    #print("File Created : {}".format(outpath))
    os.remove(pather)

In [ ]:
for dirpathed,dirnamed,file in os.walk("/content/Input"):
   z=0
   for class_ in dirnamed:
        for dirpath,dirname,files in os.walk(os.path.join(dirpathed,class_)):
              for i in files:
                    pather=os.path.join(dirpath,i)
                    print(pather)

In [ ]:
os.removedirs("/content/Input/40. I/.ipynb_checkpoints")

In [ ]:
for dirpathed,dirnamed,file in os.walk("/content/Input"):
    for class_  in dirnamed:
      os.makedirs(os.path.join("/content/Output",class_))

In [ ]:
z=0
for dirpathed,dirnamed,file in os.walk("/content/Input"):
 for class_  in dirnamed:
   for dirpath,dirname,files in os.walk(os.path.join(dirpathed,class_)):
     for i in files:
          pather=os.path.join(dirpath,i)
          z+=1
          print("..{} th Video..".format(z))
          video_array_maker(pather)
   print(".............{} over..........".format(class_))

..1 th Video..
..2 th Video..
..3 th Video..
..4 th Video..
..5 th Video..
..6 th Video..
..7 th Video..
..8 th Video..
..9 th Video..
..10 th Video..
..11 th Video..
..12 th Video..
..13 th Video..
..14 th Video..
..15 th Video..
..16 th Video..
..17 th Video..
..18 th Video..
..19 th Video..
..20 th Video..
..21 th Video..
.............45. we over..........
..22 th Video..
..23 th Video..
..24 th Video..
..25 th Video..
..26 th Video..
..27 th Video..
..28 th Video..
..29 th Video..
..30 th Video..
..31 th Video..
..32 th Video..
..33 th Video..
..34 th Video..
..35 th Video..
..36 th Video..
..37 th Video..
..38 th Video..
..39 th Video..
..40 th Video..
..41 th Video..
..42 th Video..
.............50. Alright over..........
..43 th Video..
..44 th Video..
..45 th Video..
..46 th Video..
..47 th Video..
..48 th Video..
..49 th Video..
..50 th Video..
..51 th Video..
..52 th Video..
..53 th Video..
..54 th Video..
..55 th Video..
..56 th Video..
..57 th Video..
..58 th Video..
..59 t

## Inserting "None" Class 

In [ ]:
os.makedirs("/content/Output/NoAction")

In [ ]:
def video_array_makeraction(pather,height=224,width=224):
  videodata = skvideo.io.vread(pather)  
  outpath=os.path.join("./Output/NoAction",os.path.split(pather)[1])
  out = cv2.VideoWriter(outpath,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (width,height))
  #start=time.time()

  with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#    arr=[]
    actualframe=len(videodata)
    #print("Actual frame {}".format(actualframe))
    if actualframe >=45:
          for i in range (actualframe):
            x=round (actualframe/(45)  * i)
            if x >=actualframe:
                    break
            else:
                frame =videodata[x]
                frame =cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA)
                results = holistic.process(frame)  

                output = pose_estimation(frame,results)
                output =cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
#                arr.append(output)
                out.write(output)               
    else:
          for i in range(actualframe):
              frame=videodata[i]
              frame=cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA)
              results = holistic.process(frame)  
              output = pose_estimation(frame,results)
              output =cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
              out.write(output)
          for i in range(45-actualframe):
              
              newframe=np.zeros(shape=(height,width,3))
              

              out.write(np.uint8(newframe))
    out.release()
    #print("File Created : {}".format(outpath))
    os.remove(pather)

In [ ]:
!unzip /content/NoAction.zip

In [ ]:
for dirpath,dirname,files in os.walk("/content/NoAction"):
  for i in files:
    pather=os.path.join(dirpath,i)
    video_array_makeraction(pather)

## Video Augmentation

In [ ]:
classes = [i.split(os.path.sep)[2] for i in glob.glob('./Output/*')]
classes

['45. we',
 '50. Alright',
 '42. he',
 '55. Thank you',
 '43. she',
 'NoAction',
 '56. Pleased',
 '47. they',
 '53. Good evening',
 '54. Good night',
 '48. Hello',
 '40. I',
 '51. Good Morning',
 '52. Good afternoon',
 '41. you',
 '49. How are you']

In [ ]:

# ImageDataGenerator
datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=10, # rotation
        width_shift_range=0.3, # horizontal shift
        height_shift_range=0.2, # vertical shift
        zoom_range=0.2, # zoom
        horizontal_flip=True, # horizontal flip
        brightness_range=[0.8,1.5]) # brightness

In [ ]:

# some global params
SIZE = (224,224)
CHANNELS = 3
NBFRAME = 45
BS = 4
# pattern to get videos and classes
glob_pattern='./Output/{classname}/*'
# for data augmentation

# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.2, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=datagen,
    use_frame_cache=False)

class 40. I, validation count: 3, train count: 15
class 41. you, validation count: 4, train count: 17
class 42. he, validation count: 4, train count: 17
class 43. she, validation count: 4, train count: 17
class 45. we, validation count: 4, train count: 17
class 47. they, validation count: 4, train count: 17
class 48. Hello, validation count: 4, train count: 17
class 49. How are you, validation count: 4, train count: 17
class 50. Alright, validation count: 4, train count: 17
class 51. Good Morning, validation count: 4, train count: 17
class 52. Good afternoon, validation count: 4, train count: 18
class 53. Good evening, validation count: 4, train count: 17
class 54. Good night, validation count: 4, train count: 17
class 55. Thank you, validation count: 4, train count: 17
class 56. Pleased, validation count: 4, train count: 17
class NoAction, validation count: 4, train count: 18
Total data: 16 classes for 272 files for train


## Training Model

In [ ]:
model=None

In [ ]:
mobilenet = tf.keras.applications.mobilenet.MobileNet(
        include_top=False,
        input_shape=(224,224,3),
        weights='imagenet')
    # Keep 9 layers to train
trainable = 14
for layer in mobilenet.layers[:-trainable]:
    layer.trainable = False
for layer in mobilenet.layers[-trainable:]:
    layer.trainable = True

In [ ]:
model=Sequential()
model.add(TimeDistributed(mobilenet ,input_shape=(45,224,224,3)))
model.add(TimeDistributed( Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False)) 
model.add(Dense(64,activation="relu"))
model.add(Dense(16,activation="softmax")) 
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=tf.keras.metrics.Accuracy())

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_12 (TimeDis (None, 45, 7, 7, 1024)    3228864   
_________________________________________________________________
time_distributed_13 (TimeDis (None, 45, 50176)         0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 128)               25756160  
_________________________________________________________________
dense_12 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_13 (Dense)             (None, 16)                1040      
Total params: 28,994,320
Trainable params: 27,359,312
Non-trainable params: 1,635,008
_________________________________________________________________


In [ ]:
valid = train.get_validation_generator()

Total data: 16 classes for 63 files for validation


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
checkpoint_filepath = '/content/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=3)
    #callbacks=model_checkpoint_callback)

Epoch 1/3
68/68 [==============================] - 141s 2s/step - loss: 6312.2564 - accuracy: 0.8137 - val_loss: 22759.5703 - val_accuracy: 0.8771
Epoch 2/3
68/68 [==============================] - 136s 2s/step - loss: 22866.2496 - accuracy: 0.8794 - val_loss: 32161.9688 - val_accuracy: 0.8771
Epoch 3/3
68/68 [==============================] - 136s 2s/step - loss: 22734.8194 - accuracy: 0.8839 - val_loss: 16226.6055 - val_accuracy: 0.8750


In [ ]:
model2=tensorflow.keras.models.load_model("/content/checkpoint")

In [ ]:
print(5)

5


In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_6 (TimeDist (None, 45, 5, 5, 2048)    21802784  
_________________________________________________________________
time_distributed_7 (TimeDist (None, 45, 51200)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               26280448  
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 16)                1040      
Total params: 48,092,528
Trainable params: 26,684,240
Non-trainable params: 21,408,288
_________________________________________________________________


## Checking out Model Performance on Unseen Data

In [ ]:
!unzip Tester.zip

In [ ]:
for i,j,k in os.walk("/content/Tester"):
  for m in k:
    print(os.path.join("/content/Tester",m))

/content/Tester/WhatsApp Video 2021-05-01 at 8.17.52 PM (1).mp4
/content/Tester/WhatsApp Video 2021-05-01 at 8.17.53 PM.mp4
/content/Tester/WhatsApp Video 2021-05-01 at 8.17.51 PM.mp4
/content/Tester/WhatsApp Video 2021-05-01 at 8.17.52 PM (2).mp4
/content/Tester/WhatsApp Video 2021-05-01 at 8.17.52 PM.mp4
/content/Tester/WhatsApp Video 2021-05-01 at 8.17.51 PM (1).mp4


In [ ]:
#os.makedirs("/content/Tester2")
def video_array_makeraction(pather,height=224,width=224):
  videodata = skvideo.io.vread(pather)  
  outpath=os.path.join("/content/Tester2",os.path.split(pather)[1])
  out = cv2.VideoWriter(outpath,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (width,height))
  #start=time.time()

  with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#    arr=[]
    actualframe=len(videodata)
    print("Actual frame {}".format(actualframe))
    if actualframe >=45:
          for i in range (actualframe):
            x=round (actualframe/(45)  * i)
            if x >=actualframe:
                    break
            else:
                frame =videodata[x]
                frame =cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA)
                results = holistic.process(frame)  

                output = pose_estimation(frame,results)
                output =cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
#                arr.append(output)
                out.write(output)               
    else:
          for i in range(actualframe):
              frame=videodata[i]
              frame=cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA)
              results = holistic.process(frame)  
              output = pose_estimation(frame,results)
              output =cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
              out.write(output)
          for i in range(45-actualframe):
              
              newframe=np.zeros(shape=(height,width,3))
              

              out.write(np.uint8(newframe))
    out.release()
    print("File Created : {}".format(outpath))
    #os.remove(pather)

In [ ]:
for i,j,k in os.walk("/content/Tester"):
  for m in k:
    pather=os.path.join("/content/Tester",m)
    video_array_makeraction(pather)

Actual frame 93
File Created : /content/Tester2/thank you 2.mp4
Actual frame 77
File Created : /content/Tester2/you.mp4
Actual frame 100
File Created : /content/Tester2/nothing.mp4
Actual frame 73
File Created : /content/Tester2/you2.mp4
Actual frame 120
File Created : /content/Tester2/thank you.mp4
Actual frame 102
File Created : /content/Tester2/he.mp4


In [ ]:
arr=[]
for i,j,k in os.walk("/content/Tester2"):

  for m in k:
    pather=os.path.join("/content/Tester2",m)
    print(pather)
    a=skvideo.io.vread(pather)
    arr.append(a)


arr=np.array(arr)

/content/Tester2/thank you 2.mp4
/content/Tester2/you.mp4
/content/Tester2/nothing.mp4
/content/Tester2/you2.mp4
/content/Tester2/thank you.mp4
/content/Tester2/he.mp4


In [ ]:
arr.shape

(6, 45, 224, 224, 3)

In [ ]:
prediction=model.predict(arr)

In [ ]:
for i in  np.argmax(prediction,axis=1):
  print(classes[i])

47. they
40. I
49. How are you
41. you
49. How are you
50. Alright


In [ ]:
np.argmax(prediction,axis=1)

array([5, 0, 7, 1, 7, 8])

In [ ]:
prediction.shape

(6, 16)

In [ ]:
prediction[5]

array([0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0086898e-14],
      dtype=float32)

In [ ]:
!unzip /content/tester3.zip

Archive:  /content/tester3.zip
   creating: tester3/
  inflating: tester3/WhatsApp Video 2021-05-01 at 9.21.08 PM (1).mp4  
  inflating: tester3/WhatsApp Video 2021-05-01 at 9.21.08 PM.mp4  
  inflating: tester3/WhatsApp Video 2021-05-01 at 9.21.09 PM.mp4  


In [ ]:
os.makedirs("/content/Tester4")
def video_array_makeraction(pather,height=224,width=224):
  videodata = skvideo.io.vread(pather)  
  outpath=os.path.join("/content/Tester4",os.path.split(pather)[1])
  out = cv2.VideoWriter(outpath,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (width,height))
  #start=time.time()

  with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#    arr=[]
    actualframe=len(videodata)
    print("Actual frame {}".format(actualframe))
    if actualframe >=45:
          for i in range (actualframe):
            x=round (actualframe/(45)  * i)
            if x >=actualframe:
                    break
            else:
                frame =videodata[x]
                frame =cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA)
                results = holistic.process(frame)  

                output = pose_estimation(frame,results)
                output =cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
#                arr.append(output)
                out.write(output)               
    else:
          for i in range(actualframe):
              frame=videodata[i]
              frame=cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA)
              results = holistic.process(frame)  
              output = pose_estimation(frame,results)
              output =cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
              out.write(output)
          for i in range(45-actualframe):
              
              newframe=np.zeros(shape=(height,width,3))
              

              out.write(np.uint8(newframe))
    out.release()
    print("File Created : {}".format(outpath))
    #os.remove(pather)

In [ ]:
for i,j,k in os.walk("/content/tester3"):
  for m in k:
    pather=os.path.join("/content/tester3",m)
    video_array_makeraction(pather)

Actual frame 101
File Created : /content/Tester4/WhatsApp Video 2021-05-01 at 9.21.09 PM.mp4
Actual frame 88
File Created : /content/Tester4/WhatsApp Video 2021-05-01 at 9.21.08 PM (1).mp4
Actual frame 71
File Created : /content/Tester4/WhatsApp Video 2021-05-01 at 9.21.08 PM.mp4


In [ ]:
arr=[]
for i,j,k in os.walk("/content/Tester4"):

  for m in k:
    pather=os.path.join("/content/Tester4",m)
    print(pather)
    a=skvideo.io.vread(pather)
    arr.append(a)


arr=np.array(arr)

/content/Tester4/WhatsApp Video 2021-05-01 at 9.21.09 PM.mp4
/content/Tester4/WhatsApp Video 2021-05-01 at 9.21.08 PM (1).mp4
/content/Tester4/WhatsApp Video 2021-05-01 at 9.21.08 PM.mp4


In [ ]:
prediction=model.predict(arr)

In [ ]:
for i in  np.argmax(prediction,axis=1):
  print(classes[i])

51. Good Morning
40. I
40. I


## Tring out Bigger Image Size

In [ ]:
traincheck=np.random.randn(3,45,360,540,3)

In [ ]:
mobilenet = tf.keras.applications.InceptionV3(
        include_top=False,
        input_shape=(360,540,3),
        weights='imagenet')
    # Keep 9 layers to train
trainable = 4
for layer in mobilenet.layers[:-trainable]:
    layer.trainable = False
for layer in mobilenet.layers[-trainable:]:
    layer.trainable = True

In [ ]:
model=Sequential()
model.add(TimeDistributed(mobilenet ,input_shape=(45,360,540,3)))
model.add(TimeDistributed( Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False)) 
model.add(Dense(64,activation="relu"))
model.add(Dense(16,activation="softmax")) 
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=tf.keras.metrics.Accuracy())

In [ ]:
model.fit(traincheck,np.array([0,1,0]))

array([0, 1, 0])